In [1]:
import random
import string

print("===== PASSWORD GENERATOR =====")

# Ask user for password length
length = int(input("Enter password length: "))

# Ask user preferences
print("\nSelect character types to include:")
print("1. Uppercase Letters")
print("2. Lowercase Letters")
print("3. Numbers")
print("4. Symbols")

choices = input("Enter choices (e.g., 1234 for all): ")

characters = ""

if '1' in choices:
    characters += string.ascii_uppercase

if '2' in choices:
    characters += string.ascii_lowercase

if '3' in choices:
    characters += string.digits

if '4' in choices:
    characters += string.punctuation

if characters == "":
    print("No character type selected!")
else:
    password = ""
    for i in range(length):
        password += random.choice(characters)

    print("\nGenerated Password:", password)


===== PASSWORD GENERATOR =====

Select character types to include:
1. Uppercase Letters
2. Lowercase Letters
3. Numbers
4. Symbols

Generated Password: },K]q_


In [4]:
pip install pyperclip

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import random
import string
import pyperclip

print("===== ADVANCED PASSWORD GENERATOR =====")

# Password length
length = int(input("Enter password length (minimum 4): "))

if length < 4:
    print("Password length must be at least 4!")
else:
    # Ensure at least one of each type
    upper = random.choice(string.ascii_uppercase)
    lower = random.choice(string.ascii_lowercase)
    digit = random.choice(string.digits)
    symbol = random.choice(string.punctuation)

    # Remaining characters
    remaining_length = length - 4
    all_characters = string.ascii_letters + string.digits + string.punctuation

    remaining = ''.join(random.choice(all_characters) for i in range(remaining_length))

    # Combine all
    password_list = list(upper + lower + digit + symbol + remaining)
    random.shuffle(password_list)
    password = ''.join(password_list)

    print("\nGenerated Password:", password)

    # Strength Check
    if length >= 12:
        print("Strength: Very Strong 💪")
    elif length >= 8:
        print("Strength: Strong 👍")
    else:
        print("Strength: Medium ⚠")

    # Save to file
    with open("passwords.txt", "a") as file:
        file.write(password + "\n")

    print("Password saved in passwords.txt")

    # Copy to clipboard
    pyperclip.copy(password)
    print("Password copied to clipboard!")


===== ADVANCED PASSWORD GENERATOR =====

Generated Password: DT1`lT]T
Strength: Strong 👍
Password saved in passwords.txt
Password copied to clipboard!


In [6]:
pip install cryptography


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   --- ------------------------------------ 0.3/3.5 MB ? eta -:--:--
   --------- ------------------------------ 0.8/3.5 MB 2.2 MB/s eta 0:00:02
   ------------ --------------------------- 1.0/3.5 MB 1.9 MB/s eta 0:00:02
   --------------- ------------------------ 1.3/3.5 MB 1.9 MB/s eta 0:00:02
   --------------- ------------------------ 1.3/3.5 MB 1.9 MB/s eta 0:00:02
   --------------- ------------------------ 1.3/3.5 MB 1.9 MB/s eta 0:00:02
   --------------- ------------------------ 1.3/3.5 MB 1.9 MB/s eta 0:00:02
   --------------- ------------------------ 1.3/3.5 MB 1.9 MB/s eta 0:00:02
   ------------------ --------------------- 1.6/3.5 MB 764.7 kB/s eta 0:00:03
   ------------------ --------------


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import tkinter as tk
from tkinter import messagebox
from cryptography.fernet import Fernet
import os

# Generate encryption key (only first time)
if not os.path.exists("key.key"):
    key = Fernet.generate_key()
    with open("key.key", "wb") as key_file:
        key_file.write(key)
else:
    with open("key.key", "rb") as key_file:
        key = key_file.read()

fernet = Fernet(key)

# Main Window
root = tk.Tk()
root.title("Secure Password Manager")
root.geometry("400x400")
root.resizable(False, False)

# ---------------- REGISTER FUNCTION ----------------
def register():
    username = user_entry.get()
    password = pass_entry.get()

    if username == "" or password == "":
        messagebox.showerror("Error", "Fields cannot be empty!")
        return

    encrypted_password = fernet.encrypt(password.encode())

    with open("users.txt", "ab") as file:
        file.write(username.encode() + b"|" + encrypted_password + b"\n")

    messagebox.showinfo("Success", "User Registered Successfully!")

# ---------------- LOGIN FUNCTION ----------------
def login():
    username = user_entry.get()
    password = pass_entry.get()

    if not os.path.exists("users.txt"):
        messagebox.showerror("Error", "No users registered!")
        return

    with open("users.txt", "rb") as file:
        for line in file.readlines():
            stored_user, stored_pass = line.strip().split(b"|")
            if stored_user.decode() == username:
                decrypted_password = fernet.decrypt(stored_pass).decode()
                if decrypted_password == password:
                    messagebox.showinfo("Success", "Login Successful!")
                    open_vault()
                    return

    messagebox.showerror("Error", "Invalid Credentials!")

# ---------------- VAULT WINDOW ----------------
def open_vault():
    vault = tk.Toplevel(root)
    vault.title("Password Vault")
    vault.geometry("350x300")

    tk.Label(vault, text="Stored Users", font=("Arial", 14)).pack(pady=10)

    if os.path.exists("users.txt"):
        with open("users.txt", "rb") as file:
            for line in file.readlines():
                stored_user, stored_pass = line.strip().split(b"|")
                decrypted_password = fernet.decrypt(stored_pass).decode()
                tk.Label(vault, text=f"User: {stored_user.decode()} | Pass: {decrypted_password}").pack()

# ---------------- GUI DESIGN ----------------
tk.Label(root, text="Secure Password Manager", font=("Arial", 16)).pack(pady=20)

tk.Label(root, text="Username").pack()
user_entry = tk.Entry(root)
user_entry.pack(pady=5)

tk.Label(root, text="Password").pack()
pass_entry = tk.Entry(root, show="*")
pass_entry.pack(pady=5)

tk.Button(root, text="Register", command=register, width=15).pack(pady=10)
tk.Button(root, text="Login", command=login, width=15).pack(pady=5)

root.mainloop()
